https://www.dataquest.io/blog/web-scraping-python-using-beautiful-soup/

# The requests library

In [10]:
import requests

url = "http://dataquestio.github.io/web-scraping-pages/simple.html"
page = requests.get(url)
print(page)
print(page.status_code)
print(page.content)

<Response [200]>
200
b'<!DOCTYPE html>\n<html>\n    <head>\n        <title>A simple example page</title>\n    </head>\n    <body>\n        <p>Here is some simple content for this page.</p>\n    </body>\n</html>'


In [9]:
page.content

b'<!DOCTYPE html>\n<html>\n    <head>\n        <title>A simple example page</title>\n    </head>\n    <body>\n        <p>Here is some simple content for this page.</p>\n    </body>\n</html>'

# Parsing a page with BeautifulSoup

In [12]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(page.content, 'html.parser')
print(soup.prettify())


<!DOCTYPE html>
<html>
 <head>
  <title>
   A simple example page
  </title>
 </head>
 <body>
  <p>
   Here is some simple content for this page.
  </p>
 </body>
</html>


In [13]:
list(soup.children)

['html', '\n', <html>
 <head>
 <title>A simple example page</title>
 </head>
 <body>
 <p>Here is some simple content for this page.</p>
 </body>
 </html>]

In [14]:
[type(item) for item in list(soup.children)]

[bs4.element.Doctype, bs4.element.NavigableString, bs4.element.Tag]

In [17]:
html = list(soup.children)
list(html.children)

['\n', <head>
 <title>A simple example page</title>
 </head>, '\n', <body>
 <p>Here is some simple content for this page.</p>
 </body>, '\n']

In [21]:
list(soup.children)[2]

<html>
<head>
<title>A simple example page</title>
</head>
<body>
<p>Here is some simple content for this page.</p>
</body>
</html>

In [24]:
body = list(html.children)[3]
list(body.children)

['\n', <p>Here is some simple content for this page.</p>, '\n']

In [25]:
p = list(body.children)[1]

In [26]:
p.get_text()

'Here is some simple content for this page.'

# Finding all instances of a tag at once

In [27]:
soup = BeautifulSoup(page.content, 'html.parser')
soup.find_all('p')

[<p>Here is some simple content for this page.</p>]

In [28]:
soup.find_all('p')[0].get_text()

'Here is some simple content for this page.'

In [29]:
soup.find('p')

<p>Here is some simple content for this page.</p>

# Searching for tags by class and id

In [32]:
url = "http://dataquestio.github.io/web-scraping-pages/ids_and_classes.html"
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
print(soup)

<html>
<head>
<title>A simple example page</title>
</head>
<body>
<div>
<p class="inner-text first-item" id="first">
                First paragraph.
            </p>
<p class="inner-text">
                Second paragraph.
            </p>
</div>
<p class="outer-text first-item" id="second">
<b>
                First outer paragraph.
            </b>
</p>
<p class="outer-text">
<b>
                Second outer paragraph.
            </b>
</p>
</body>
</html>


In [33]:
soup.find_all('p', class_='outer-text')

[<p class="outer-text first-item" id="second">
 <b>
                 First outer paragraph.
             </b>
 </p>, <p class="outer-text">
 <b>
                 Second outer paragraph.
             </b>
 </p>]

In [34]:
soup.find_all(class_="outer-text")

[<p class="outer-text first-item" id="second">
 <b>
                 First outer paragraph.
             </b>
 </p>, <p class="outer-text">
 <b>
                 Second outer paragraph.
             </b>
 </p>]

In [35]:
soup.find_all(id="first")

[<p class="inner-text first-item" id="first">
                 First paragraph.
             </p>]

# Using CSS Selectors
- p a — finds all a tags inside of a p tag.
- body p a — finds all a tags inside of a p tag inside of a body tag.
- html body — finds all body tags inside of an html tag.
- p.outer-text — finds all p tags with a class of outer-text.
- p#first — finds all p tags with an id of first.
- body p.outer-text — finds any p tags with a class of outer-text inside of a body tag.

In [36]:
soup.select("div p")

[<p class="inner-text first-item" id="first">
                 First paragraph.
             </p>, <p class="inner-text">
                 Second paragraph.
             </p>]

# Downloading weather data

- Download the web page containing the forecast.  
- Create a BeautifulSoup class to parse the page.  
- Find the div with id seven-day-forecast, and assign to seven_day  
- Inside seven_day, find each individual forecast item.
Extract and print the first forecast item.

In [38]:
url = "http://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168"
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_="tombstone-container")
tonight = forecast_items[0]
print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Today
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Today: Sunny, with a high near 80. Calm wind becoming west 5 to 9 mph in the afternoon. " class="forecast-icon" src="newimages/medium/skc.png" title="Today: Sunny, with a high near 80. Calm wind becoming west 5 to 9 mph in the afternoon. "/>
 </p>
 <p class="short-desc">
  Sunny
 </p>
 <p class="temp temp-high">
  High: 80 °F
 </p>
</div>


# Extracting information from the page

In [39]:
period = tonight.find(class_="period-name").get_text()
short_desc = tonight.find(class_="short-desc").get_text()
temp = tonight.find(class_="temp").get_text()
print(period)
print(short_desc)
print(temp)

Today
Sunny
High: 80 °F


In [40]:
img = tonight.find("img")
desc = img['title']
print(desc)

Today: Sunny, with a high near 80. Calm wind becoming west 5 to 9 mph in the afternoon. 


# Extracting all the information from the page

In [42]:
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
print(periods)

['Today', 'Tonight', 'Friday', 'FridayNight', 'Saturday', 'SaturdayNight', 'Sunday', 'SundayNight', 'Monday']


In [49]:
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]
print(short_descs)
print(temps)
print(descs)

['Sunny', 'Mostly Clear', 'Mostly Sunny', 'Partly Cloudy', 'Partly Sunny', 'Mostly Cloudy', 'Mostly Sunny', 'Slight ChanceRain', 'Chance Rain']
['High: 80 °F', 'Low: 48 °F', 'High: 67 °F', 'Low: 48 °F', 'High: 62 °F', 'Low: 50 °F', 'High: 63 °F', 'Low: 50 °F', 'High: 62 °F']
['Today: Sunny, with a high near 80. Calm wind becoming west 5 to 9 mph in the afternoon. ', 'Tonight: Mostly clear, with a low around 48. West wind 5 to 10 mph becoming light west southwest  after midnight. ', 'Friday: Mostly sunny, with a high near 67. Calm wind becoming west 5 to 9 mph in the afternoon. ', 'Friday Night: Partly cloudy, with a low around 48. West wind 5 to 10 mph. ', 'Saturday: Partly sunny, with a high near 62. West southwest wind 6 to 13 mph. ', 'Saturday Night: Mostly cloudy, with a low around 50.', 'Sunday: Mostly sunny, with a high near 63.', 'Sunday Night: A slight chance of rain.  Partly cloudy, with a low around 50.', 'Monday: A chance of rain.  Partly sunny, with a high near 62.']


# Combining our data into a Pandas Dataframe

In [50]:
import pandas as pd

weather = pd.DataFrame({
    "period": periods,
    "short_desc": short_descs,
    "temp": temps,
    "desc":descs
})
weather

,period,short_desc,temp,desc
0,Today,Sunny,High: 80 °F,"Today: Sunny, with a high near 80. Calm wind b..."
1,Tonight,Mostly Clear,Low: 48 °F,"Tonight: Mostly clear, with a low around 48. W..."
2,Friday,Mostly Sunny,High: 67 °F,"Friday: Mostly sunny, with a high near 67. Cal..."
3,FridayNight,Partly Cloudy,Low: 48 °F,"Friday Night: Partly cloudy, with a low around..."
4,Saturday,Partly Sunny,High: 62 °F,"Saturday: Partly sunny, with a high near 62. W..."
5,SaturdayNight,Mostly Cloudy,Low: 50 °F,"Saturday Night: Mostly cloudy, with a low arou..."
6,Sunday,Mostly Sunny,High: 63 °F,"Sunday: Mostly sunny, with a high near 63."
7,SundayNight,Slight ChanceRain,Low: 50 °F,Sunday Night: A slight chance of rain. Partly...
8,Monday,Chance Rain,High: 62 °F,"Monday: A chance of rain. Partly sunny, with ..."


- str 참조 : https://yganalyst.github.io/data_handling/memo_9/
- 정규식 참조 : https://wikidocs.net/4308

In [74]:
temp_nums = weather["temp"].str.extract("(\d+)", expand=False)
weather["temp_num"] = temp_nums.astype('int')
print(temp_nums)

0    80
1    48
2    67
3    48
4    62
5    50
6    63
7    50
8    62
Name: temp, dtype: object


In [75]:
weather["temp_num"].mean()

58.888888888888886

In [77]:
is_night = weather["temp"].str.contains("Low")
weather["is_night"] = is_night
print(is_night)

0    False
1     True
2    False
3     True
4    False
5     True
6    False
7     True
8    False
Name: temp, dtype: bool


In [82]:
weather[is_night]

,period,short_desc,temp,desc,temp_num,is_night
1,Tonight,Mostly Clear,Low: 48 °F,"Tonight: Mostly clear, with a low around 48. W...",48,True
3,FridayNight,Partly Cloudy,Low: 48 °F,"Friday Night: Partly cloudy, with a low around...",48,True
5,SaturdayNight,Mostly Cloudy,Low: 50 °F,"Saturday Night: Mostly cloudy, with a low arou...",50,True
7,SundayNight,Slight ChanceRain,Low: 50 °F,Sunday Night: A slight chance of rain. Partly...,50,True
